In [2]:
# Import pandas & yfinance
import pandas as pd
import yfinance as yf

# Import numpy
import numpy as np
from numpy import *
from numpy.linalg import multi_dot

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600))) # theme= 'henanigans'

# Import plotly express for EF plot
import plotly.express as px
# px.defaults.template = "plotly_dark"
px.defaults.width, px.defaults.height = 1000, 600

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

In [3]:
# Specify assets / stocks
# international etf portfolio : 'SPY', 'GLD', 'IWM', 'VWO', 'BND']
# indian stocks : bank, consumer goods, diversified, it, consumer durables
assets = ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets.sort()

# Number of assets
numofasset = len(assets)

# Number of portfolio for optimization
numofportfolio = 5000

In [5]:
# Get yahoo tickers
yahooticker = [x+'.NS' for x in assets]
# Fetch / read data for multiple stocks at once
df = yf.download(yahooticker, start='2015-01-01', end='2022-12-31', progress=False)['Adj Close']
df.columns = assets

# write data to file for future use
df.to_csv('india_stocks.csv')

# Read from file
# df = pd.read_csv('data/india_stocks.csv', index_col=0, parse_dates=True)

# Display dataframe
df


,ASIANPAINT,HDFCBANK,ITC,RELIANCE,TCS
Date,,,,,
2015-01-01,709.6138,451.4753,192.3574,417.9889,1075.8508
2015-01-02,734.1468,457.7586,192.9599,416.8826,1090.1786
2015-01-05,734.0995,453.8938,193.7196,412.3163,1073.6110
2015-01-06,716.5759,446.8280,188.7424,393.6035,1034.0303
2015-01-07,730.9737,448.1321,185.2321,402.1714,1021.8165
...,...,...,...,...,...
2022-12-26,3056.0500,1629.4500,329.4050,2524.0500,3188.3152
2022-12-27,3112.6001,1631.1000,327.9775,2544.7000,3194.7844
2022-12-28,3123.7000,1629.8000,329.2081,2544.4500,3192.4321


In [6]:
# Plot price history
df['2022':].normalize().iplot(kind='line', title='Normalized Price Plot')

In [7]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
df1 = pd.DataFrame({
    'Ann Ret': round(returns.mean()*252*100,2),
    'Ann Vol': round(returns.std()*np.sqrt(252)*100,2)
})
df1

,Ann Ret,Ann Vol
ASIANPAINT,22.27,26.46
HDFCBANK,19.10,23.38
ITC,10.30,26.58
RELIANCE,27.36,29.35
TCS,16.85,24.39


In [8]:
# Plot annualized return and volatility
df1.iplot(
    kind='bar', 
    title='Annualized Return & Volatility (%)', 
    shared_xaxes = True,  
    orientation="h") #subplots=True

In [9]:
df1.reset_index().iplot(
    kind="pie", 
    title='Annualized Return (%)', 
    labels='index', 
    values="Ann Ret",
    textinfo='percent+label', 
    hole=.4)

In [10]:
def portfolio_simulation(returns):

    # Initialize the lists
    rets = []; vols = []; wts = []

    # Simulate 5,000 portfolios
    for i in range (numofportfolio):
        
        # Generate random weights
        weights = random.random(numofasset)[:, newaxis]
        
        # Set weights such that sum of weights equals 1
        weights /= sum(weights)
        
        # Portfolio statistics
        rets.append(weights.T @ array(returns.mean() * 252)[:, newaxis])        
        vols.append(sqrt(multi_dot([weights.T, returns.cov()*252, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
        })
    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf,2)


In [11]:
# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,weights,sharpe_ratio
0,20.80,17.46,"[24.077474120706214, 23.188479153511434, 5.207...",1.19
1,16.98,18.12,"[8.179458762129306, 1.257823863523624, 41.5824...",0.94
2,17.59,17.09,"[10.707430418159667, 13.176791125592782, 31.94...",1.03
3,17.98,18.21,"[13.722286925144195, 28.431474548073975, 36.12...",0.99
4,21.65,19.00,"[8.039861390585914, 16.568233877244534, 5.7486...",1.14


In [12]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   21.71
port_vols                                                   18.16
weights         [28.96248042919064, 21.66954412273109, 3.06529...
sharpe_ratio                                                  1.2
Name: 53, dtype: object

In [13]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,19.1927,1.6207,13.09,18.12,19.18,20.29,24.99
port_vols,5000.0,17.7962,0.8670,16.50,17.17,17.62,18.23,24.14
sharpe_ratio,5000.0,1.0788,0.0778,0.61,1.04,1.09,1.14,1.20


In [14]:
# Max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# Allocation to achieve max sharpe ratio portfolio
dict(zip(assets, around(msrpwts,2)))

{'ASIANPAINT': 28.96,
 'HDFCBANK': 21.67,
 'ITC': 3.07,
 'RELIANCE': 28.56,
 'TCS': 17.74}

In [15]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio', 
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 
fig.add_scatter(
    mode='markers', 
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']], 
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']], 
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [16]:
# Import optimization module from scipy
import scipy.optimize as sco

In [17]:
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights])) 
    
    return np.array([port_rets, port_vols, port_rets/port_vols]).flatten()


# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [18]:
# Specify constraints, bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in range(numofasset))
initial_wts = numofasset*[1./numofasset]

In [19]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [20]:
# Efficient Frontier
# targetrets = linspace(0.01,0.11,100)
targetrets = linspace(0.155,0.24,100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = array(tvols)

In [21]:
# Create EF Dataframe for plotting
efport = pd.DataFrame({
    'targetrets' : around(100*targetrets,2),
    'targetvols': around(100*targetvols,2),
    'targetsharpe': around(targetrets/targetvols,2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,15.50,17.15,0.90
1,15.59,17.09,0.91
2,15.67,17.04,0.92
3,15.76,16.99,0.93
4,15.84,16.94,0.94


In [22]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()